# Notebook 01: Problem Statement and Data Cleaning

This notebook details the problem statement and cleans/saves the datasets.

## Problem Statement

[CTE Watch Company](https://www.cte-watches.com/who-we-are) is a South Florida based  company that distributes fashion and household items to smaller retailers in the Caribbean islands, Cruise ships, Duty-free/Travel Retail, Latin America & North American markets.

Success is determined by how well CTE manages inventory to optimize warehouse space and minimize the purchase of unsuccessful products. CTE typically purchases directly from the brands on a quarterly basis, which they then go on to sell to the smaller retailers. They must  predict how much they can sell to the smaller retailers in order to place quarterly brand orders. Typically, CTE makes their predictions by calculating a 6-month rolling average and placing an order to make sure they have enough inventory for 3 months. If it is a new model, they must make a guess based on the attributes of the watch. This is a very manual process and requires the 30-year expertise of the owner to be done well. Additionally, there are many different types of products within each brand (ex. 300 different watch models for 1 brand), and there are approximately 40 brands.

In this project, I aim to build a model that accurately and more quickly forecasts order quantities for both new and existing models for one watch brand. The model will predict watch sales based on historical sales (if available) and the individual model attributes (style, gender, color, price, material, etc). Different types of models will be explored to adequately and appropriately accomplish this goal (forecasting, regression, classification). This predictions will be validated against the owner's purchase order for the next quarter during a meeting in early June.

## Data Cleaning